In [2]:
from transformers import pipeline
import torch

In [3]:
MODEL = "NbAiLab/whisper-large-sme"
LANG = "fi"

In [4]:
if torch.cuda.is_available():
    device = 0
else:
    device = "cpu"

In [ ]:
pipe = pipeline(task="automatic-speech-recognition", model=MODEL, chunk_length_s=30, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

In [ ]:
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

In [2]:
PAGE = "https://sverigesradio.se/artikel/odda-skearru-bitonsami-ludiiguin"
AUDIOJSON = "https://sverigesradio.se/playerajax/audio?id=8580562&type=publication&publicationid=8580562&quality=medium"

In [3]:
import requests
import json
def get_sverigesradio_audio(page):
    req = requests.get(page)
    if req.status_code != 200:
        return None
    data = json.loads(req.text)
    if "audioUrl" in data:
        return data["audioUrl"]
    return None

In [ ]:
!wget {get_sverigesradio_audio(AUDIOJSON)}

In [6]:
from bs4 import BeautifulSoup

In [7]:
def get_audio_id_from_page(page):
    req = requests.get(page)
    if req.status_code != 200:
        return None
    soup = BeautifulSoup(req.text, 'html.parser')
    for elem in soup.findAll("script", {"id": "gtm-metadata"}):
        if "pageId" in elem.text:
            data = json.loads(elem.text)
            return data["pageId"]
    return None

In [8]:
get_audio_id_from_page(PAGE)

'8580562'

In [9]:
def get_audio_from_page(page):
    pageid = get_audio_id_from_page(page)
    if pageid is None:
        return None
    return get_sverigesradio_audio(f"https://sverigesradio.se/playerajax/audio?id={pageid}&type=publication&publicationid={pageid}&quality=medium")

In [10]:
get_audio_from_page(PAGE)

'https://lyssna-cdn.sr.se/isidor/2024/02/program_odda_bihtasmiluodii_skearrus_20240209_1144255925_a96.m4a?_h_publicationId=8580562'